# イールドカーブ主成分分析（最新100営業日）

## 概要
日本国債のイールドカーブデータに対して主成分分析（PCA）を実行します。

## 分析手順
1. 最新日から過去100営業日分のデータを取得
2. 全日付の残存期間の和集合を作成（0.5年未満を除外）
3. 3次スプライン補間で欠損値を補完
4. 主成分分析を実行
5. 結果を可視化・解釈

## パラメータ
- **分析期間**: 最新100営業日
- **残存期間閾値**: 0.5年以上
- **補間方法**: 3次スプライン補間

## 1. 環境設定とライブラリのインポート

In [ ]:
import os
import sys
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import CubicSpline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from dotenv import load_dotenv

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

# 環境変数読み込み
load_dotenv(os.path.join(project_root, '.env'))

# Supabase設定
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')

# グラフ設定
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

print('✅ 環境設定完了')
print(f'Supabase URL: {SUPABASE_URL[:40]}...' if SUPABASE_URL else '❌ SUPABASE_URL not set')
print(f'Supabase Key: {SUPABASE_KEY[:20]}...' if SUPABASE_KEY else '❌ SUPABASE_KEY not set')

## 2. データ取得関数

In [ ]:
def get_latest_dates(limit=200):
    """
    最新の営業日を取得
    
    Parameters:
    -----------
    limit : int
        取得する日付数（100営業日を確保するため余裕を持たせる）
    
    Returns:
    --------
    list : 日付リスト（降順）
    """
    headers = {
        'apikey': SUPABASE_KEY,
        'Authorization': f'Bearer {SUPABASE_KEY}'
    }
    
    response = requests.get(
        f'{SUPABASE_URL}/rest/v1/bond_data',
        params={
            'select': 'trade_date',
            'order': 'trade_date.desc',
            'limit': limit * 10  # データが重複しているため多めに取得
        },
        headers=headers
    )
    
    if response.status_code != 200:
        raise Exception(f'データ取得エラー: {response.status_code}')
    
    # ユニークな日付を抽出してソート
    dates = sorted(list(set([row['trade_date'] for row in response.json()])), reverse=True)
    
    return dates[:limit]


def get_yield_data_for_date(date):
    """
    指定日のイールドカーブデータを取得
    
    Parameters:
    -----------
    date : str
        取得日付（YYYY-MM-DD）
    
    Returns:
    --------
    pd.DataFrame : カラム [maturity_years, yield_rate]
    """
    headers = {
        'apikey': SUPABASE_KEY,
        'Authorization': f'Bearer {SUPABASE_KEY}'
    }
    
    response = requests.get(
        f'{SUPABASE_URL}/rest/v1/bond_data',
        params={
            'select': 'trade_date,due_date,ave_compound_yield',
            'trade_date': f'eq.{date}',
            'ave_compound_yield': 'not.is.null',
            'due_date': 'not.is.null'
        },
        headers=headers
    )
    
    if response.status_code != 200 or not response.json():
        return pd.DataFrame(columns=['maturity_years', 'yield_rate'])
    
    # 残存年数を計算
    data_list = []
    for row in response.json():
        try:
            trade_dt = datetime.strptime(row['trade_date'], '%Y-%m-%d')
            due_dt = datetime.strptime(row['due_date'], '%Y-%m-%d')
            maturity_years = (due_dt - trade_dt).days / 365.25
            
            # 0.5年以上のデータのみ
            if maturity_years >= 0.5 and row['ave_compound_yield'] is not None:
                data_list.append({
                    'maturity_years': round(maturity_years, 4),
                    'yield_rate': float(row['ave_compound_yield'])
                })
        except (ValueError, TypeError):
            continue
    
    return pd.DataFrame(data_list)


print('✅ データ取得関数定義完了')

## 3. データ取得

In [ ]:
# 最新の営業日を取得
print('📅 営業日取得中...')
available_dates = get_latest_dates(limit=200)
target_dates = available_dates[:100]  # 最新100営業日

print(f'✅ 分析対象日付: {len(target_dates)}日')
print(f'   期間: {target_dates[-1]} ~ {target_dates[0]}')
print(f'   最新日: {target_dates[0]}')

In [ ]:
# 各日付のデータを取得
print('\n📊 イールドカーブデータ取得中...')
daily_data = {}

for i, date in enumerate(target_dates, 1):
    df = get_yield_data_for_date(date)
    if not df.empty:
        daily_data[date] = df
    
    if i % 20 == 0:
        print(f'   進捗: {i}/{len(target_dates)}日')

print(f'\n✅ データ取得完了: {len(daily_data)}日分')

# データ統計
total_points = sum(len(df) for df in daily_data.values())
avg_points = total_points / len(daily_data) if daily_data else 0

print(f'   総データポイント数: {total_points:,}個')
print(f'   1日あたり平均: {avg_points:.1f}個')

## 4. 残存期間の和集合作成とスプライン補間

In [ ]:
def create_maturity_union(daily_data):
    """
    全日付の残存期間の和集合を作成
    
    Parameters:
    -----------
    daily_data : dict
        日付ごとのDataFrame辞書
    
    Returns:
    --------
    np.ndarray : ソート済み残存期間配列
    """
    all_maturities = set()
    
    for df in daily_data.values():
        all_maturities.update(df['maturity_years'].values)
    
    return np.sort(np.array(list(all_maturities)))


def interpolate_yields(df, common_grid):
    """
    3次スプライン補間で共通年限軸上の利回りを推定
    
    Parameters:
    -----------
    df : pd.DataFrame
        ある日付のデータ
    common_grid : np.ndarray
        共通残存期間軸
    
    Returns:
    --------
    np.ndarray : 補間後の利回り配列
    """
    if df.empty or len(df) < 2:
        return np.full(len(common_grid), np.nan)
    
    # 重複削除・ソート
    df_sorted = df.sort_values('maturity_years').drop_duplicates('maturity_years')
    
    if len(df_sorted) < 2:
        return np.full(len(common_grid), np.nan)
    
    # 3次スプライン補間
    cs = CubicSpline(
        df_sorted['maturity_years'].values,
        df_sorted['yield_rate'].values,
        extrapolate=False  # 外挿なし
    )
    
    return cs(common_grid)


print('✅ スプライン補間関数定義完了')

In [ ]:
# 残存期間の和集合を作成
print('\n🔗 残存期間の和集合作成中...')
common_maturities = create_maturity_union(daily_data)

print(f'✅ 共通残存期間軸: {len(common_maturities)}個')
print(f'   範囲: {common_maturities.min():.2f}年 ~ {common_maturities.max():.2f}年')

# 分布確認
short_term = np.sum(common_maturities <= 2)
medium_term = np.sum((common_maturities > 2) & (common_maturities <= 10))
long_term = np.sum(common_maturities > 10)

print(f'\n   期間別分布:')
print(f'     短期（≤2年）: {short_term}個')
print(f'     中期（2-10年）: {medium_term}個')
print(f'     長期（>10年）: {long_term}個')

In [ ]:
# スプライン補間でデータ行列を作成
print('\n🔄 スプライン補間実行中...')

interpolated_data = []
valid_dates = []

for i, (date, df) in enumerate(daily_data.items(), 1):
    interpolated = interpolate_yields(df, common_maturities)
    
    # NaNが50%未満の行のみ採用
    nan_ratio = np.isnan(interpolated).sum() / len(interpolated)
    if nan_ratio < 0.5:
        interpolated_data.append(interpolated)
        valid_dates.append(date)
    
    if i % 20 == 0:
        print(f'   進捗: {i}/{len(daily_data)}日')

# NumPy配列に変換
X = np.array(interpolated_data)

print(f'\n✅ スプライン補間完了')
print(f'   有効日数: {len(valid_dates)}日')
print(f'   データ行列サイズ: {X.shape}（日数 x 残存期間）')
print(f'   NaN含有率: {np.isnan(X).sum() / X.size * 100:.2f}%')

## 5. 主成分分析（PCA）実行

In [ ]:
# NaNを列平均で補完
print('\n🔧 NaN補完中...')
X_filled = X.copy()
col_means = np.nanmean(X, axis=0)

for i in range(X.shape[1]):
    mask = np.isnan(X_filled[:, i])
    X_filled[mask, i] = col_means[i]

print(f'✅ NaN補完完了')
print(f'   補完後のNaN数: {np.isnan(X_filled).sum()}個（0であるべき）')

In [ ]:
# データ標準化
print('\n📊 データ標準化中...')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)

print(f'✅ 標準化完了')
print(f'   平均: {X_scaled.mean():.6f} (≈0)')
print(f'   標準偏差: {X_scaled.std():.6f} (≈1)')

In [ ]:
# PCA実行
print('\n🎯 主成分分析実行中...')
n_components = min(10, X_scaled.shape[0], X_scaled.shape[1])  # 最大10主成分

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

print(f'✅ PCA完了')
print(f'   主成分数: {pca.n_components_}個')
print(f'   変換後データサイズ: {X_pca.shape}')
print(f'\n📈 寄与率:')
for i, ratio in enumerate(pca.explained_variance_ratio_[:5], 1):
    print(f'   PC{i}: {ratio*100:.2f}%')

cumsum = np.cumsum(pca.explained_variance_ratio_)
print(f'\n📊 累積寄与率:')
print(f'   PC1-3: {cumsum[2]*100:.2f}%')
print(f'   PC1-5: {cumsum[4]*100:.2f}%' if len(cumsum) >= 5 else '')

## 6. 結果の可視化

In [ ]:
# 1. 寄与率のスクリープロット
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 個別寄与率
axes[0].bar(range(1, len(pca.explained_variance_ratio_) + 1), 
            pca.explained_variance_ratio_ * 100,
            alpha=0.7,
            color='steelblue')
axes[0].set_xlabel('主成分', fontsize=12)
axes[0].set_ylabel('寄与率 (%)', fontsize=12)
axes[0].set_title('主成分ごとの寄与率', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 累積寄与率
axes[1].plot(range(1, len(cumsum) + 1), 
             cumsum * 100, 
             marker='o', 
             linewidth=2,
             markersize=8,
             color='darkred')
axes[1].axhline(y=90, color='gray', linestyle='--', linewidth=1, alpha=0.7, label='90%')
axes[1].axhline(y=95, color='gray', linestyle='--', linewidth=1, alpha=0.7, label='95%')
axes[1].set_xlabel('主成分数', fontsize=12)
axes[1].set_ylabel('累積寄与率 (%)', fontsize=12)
axes[1].set_title('累積寄与率', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ 寄与率グラフ表示完了')

In [ ]:
# 2. 主成分ベクトル（固有ベクトル）の可視化
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

colors = ['#e74c3c', '#3498db', '#2ecc71']
labels = ['PC1（水準変動）', 'PC2（傾き変動）', 'PC3（曲率変動）']

for i in range(3):
    axes[i].plot(common_maturities, 
                 pca.components_[i], 
                 linewidth=2.5,
                 color=colors[i],
                 label=labels[i])
    axes[i].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    axes[i].set_xlabel('残存期間（年）', fontsize=12)
    axes[i].set_ylabel('固有ベクトル', fontsize=12)
    axes[i].set_title(f'{labels[i]} - 寄与率: {pca.explained_variance_ratio_[i]*100:.2f}%', 
                      fontsize=13, 
                      fontweight='bold')
    axes[i].legend(loc='best', fontsize=11)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ 主成分ベクトル表示完了')

In [ ]:
# 3. 主成分スコアの時系列変化
fig, axes = plt.subplots(3, 1, figsize=(16, 10))

# 日付をdatetimeに変換
date_objects = [datetime.strptime(d, '%Y-%m-%d') for d in valid_dates]

for i in range(3):
    axes[i].plot(date_objects, 
                 X_pca[:, i], 
                 linewidth=2,
                 color=colors[i],
                 alpha=0.8)
    axes[i].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    axes[i].set_ylabel(f'PC{i+1} スコア', fontsize=12)
    axes[i].set_title(f'{labels[i]}の時系列変化', fontsize=13, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    
    # x軸を回転
    plt.setp(axes[i].xaxis.get_majorticklabels(), rotation=45)

axes[-1].set_xlabel('日付', fontsize=12)

plt.tight_layout()
plt.show()

print('✅ 主成分スコア時系列グラフ表示完了')

## 7. 結果の解釈

In [ ]:
print('=' * 70)
print('📋 主成分分析結果サマリー')
print('=' * 70)
print(f'分析期間: {valid_dates[-1]} ~ {valid_dates[0]}')
print(f'日数: {len(valid_dates)}日')
print(f'残存期間軸: {len(common_maturities)}個（{common_maturities.min():.2f}年 ~ {common_maturities.max():.2f}年）')
print()
print('主成分の寄与率:')
for i in range(min(5, len(pca.explained_variance_ratio_))):
    print(f'  PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}% （累積: {cumsum[i]*100:.2f}%）')
print()
print('=' * 70)
print('💡 解釈')
print('=' * 70)
print('PC1（第1主成分）: イールドカーブの水準変動（パラレルシフト）')
print('  → 全期間の金利が一斉に上下する動き')
print()
print('PC2（第2主成分）: イールドカーブの傾き変動（スティープ化/フラット化）')
print('  → 短期金利と長期金利の差が変化する動き')
print()
print('PC3（第3主成分）: イールドカーブの曲率変動（バタフライ）')
print('  → 中期金利が短期・長期に対して相対的に変化する動き')
print('=' * 70)

## 8. データのエクスポート（オプション）

In [ ]:
# 主成分スコアをDataFrameに変換して保存
pca_scores_df = pd.DataFrame(
    X_pca,
    columns=[f'PC{i+1}' for i in range(X_pca.shape[1])],
    index=valid_dates
)
pca_scores_df.index.name = 'trade_date'

# 表示
print('主成分スコア（最新10日分）:')
print(pca_scores_df.head(10))

# CSV出力（オプション）
# output_path = os.path.join(project_root, 'data_files', 'pca_scores.csv')
# pca_scores_df.to_csv(output_path)
# print(f'\n✅ 主成分スコアを保存: {output_path}')

## 9. まとめ

このノートブックでは、日本国債のイールドカーブデータに対して主成分分析を実行しました。

### 実施内容
1. ✅ 最新100営業日分のデータを取得
2. ✅ 残存期間0.5年未満を除外
3. ✅ 全日付の残存期間の和集合を作成
4. ✅ 3次スプライン補間で欠損値を補完
5. ✅ 主成分分析を実行
6. ✅ 寄与率、固有ベクトル、主成分スコアを可視化

### 主な発見
- **PC1（水準変動）**: イールドカーブ全体の上下動を表現
- **PC2（傾き変動）**: 短期と長期の金利差の変化を表現
- **PC3（曲率変動）**: 中期金利の相対的な動きを表現

これらの主成分は、金利リスク管理やポートフォリオ最適化に活用できます。